In [417]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.use('TkAgg')

In [416]:
dataframe1_sorb = pd.read_excel('Silica-loc-isoth1.xlsx', header=None, sheet_name="Adsorption")
data_sorb = dataframe1_sorb.to_numpy()
dataframe1_desorb = pd.read_excel('Silica-loc-isoth1.xlsx', header=None, sheet_name="Desorption")
data_desorb = dataframe1_desorb.to_numpy()

In [418]:
pressure_start_index = 21

pore_sizes_s = data_sorb[0][1:]
pressures_s = data_sorb[:, 0][pressure_start_index:]

pore_sizes_d = data_desorb[0][1:]
pressures_d = data_desorb[:, 0][pressure_start_index:]

In [419]:
N = data_sorb.shape[0]
P_N = len(pressures_s)
n_s = np.zeros(len(pressures_s))
n_d = np.zeros(len(pressures_d))

In [420]:
sigma1 = 2
sigma2 = 5
d0_1 = 9
d0_2 = 20

pore_distribution = (1 / sigma1) * np.exp(- np.power((pore_sizes_d - d0_1), 2) / (2 * sigma1 ** 2))
pore_distribution += (1 / sigma2) * np.exp(- np.power((pore_sizes_d - d0_2), 2) / (2 * sigma2 ** 2))

In [421]:
for p_i in range(pressure_start_index, N):
    for d_i in range(len(pore_distribution)):
        n_s[p_i - pressure_start_index] += pore_distribution[d_i] * data_sorb[p_i][d_i]

for p_i in range(pressure_start_index, N):
    for d_i in range(len(pore_distribution)):
        n_d[p_i - pressure_start_index] += pore_distribution[d_i] * data_desorb[p_i][d_i]

In [422]:
for i in range(1, P_N-1):
    n_s[i] = (n_s[i-1] + n_s[i] + n_s[i+1]) / 3
    n_d[i] = (n_d[i-1] + n_d[i] + n_d[i+1]) / 3

In [423]:
max_value = max(n_s.max(), n_d.max())
n_s = n_s / max_value
n_d = n_d / max_value

In [434]:
# figure, axis = plt.subplots(1, 2)
# axis[0].plot(pore_sizes_d, pore_distribution, marker=".")
# axis[1].plot(pressures_s, n_s, marker=".")
# axis[1].plot(pressures_d, n_d, marker=".")
plt.plot(pressures_d, n_d, marker=".")
plt.plot(pressures_d, n_s, marker=".")
plt.show()

In [430]:
resolution = 100
picture = np.zeros((resolution, resolution))

In [431]:
def bresenham_line(x0, y0, x1, y1):
    steep = abs(y1 - y0) > abs(x1 - x0)
    if steep:
        x0, y0 = y0, x0
        x1, y1 = y1, x1

    switched = False
    if x0 > x1:
        switched = True
        x0, x1 = x1, x0
        y0, y1 = y1, y0

    if y0 < y1:
        ystep = 1
    else:
        ystep = -1

    deltax = x1 - x0
    deltay = abs(y1 - y0)
    error = -deltax / 2
    y = y0

    line = []
    for x in range(x0, x1 + 1):
        if steep:
            line.append((y,x))
        else:
            line.append((x,y))

        error = error + deltay
        if error > 0:
            y = y + ystep
            error = error - deltax
    if switched:
        line.reverse()
    return line

In [432]:
def graph_to_picture(n_array, pressure_array, resolution, picture):
    tmp_x, tmp_y = None, None
    for p_i in range(len(pressure_array)):
            x = int(n_array[p_i] * (resolution-1))
            y = int(pressure_array[p_i] * (resolution-1))
            picture[x][y] = 1
            if tmp_x is not None:  # connect points with line
                line = bresenham_line(tmp_x, tmp_y, x, y)
                for a, b in line:
                    picture[a][b] = 1
            tmp_x, tmp_y = x, y
graph_to_picture(n_s, pressures_s, picture)
graph_to_picture(n_d, pressures_d, picture)

In [433]:
fig = plt.figure
plt.imshow(picture, cmap='gray', origin='lower')
plt.show()